In [53]:
import datetime
import os
import sys
import tensorflow as tf
import json
from data_process import data_process
import pandas as pd

TYPE_DICT = {'string': tf.string, 'int64': tf.int64, 'float32': tf.float32}

In [54]:
def _parse_example(example_proto):
    features = tf.io.parse_single_example(example_proto, feature_description)
    return features

In [55]:
train_file, test_file = data_process.mac_train_test_file()
# feature 文件
current_dir = %pwd
json_file = os.path.join(current_dir, 'feature.json')
all_features = json.load(open(json_file))

feature_description = {}
for key, value in all_features.items():
    feature_description[key] = tf.io.FixedLenFeature(shape=(value[1],), dtype=TYPE_DICT[value[0]])

dataset = tf.data.TFRecordDataset(train_file)
[feature] = dataset.take(1)
print(feature)

添加的文件数量：训练集 4 测试集 0
tf.Tensor(b'\n\xfe\x13\n\x11\n\x08week_day\x12\x05\x1a\x03\n\x01\x01\n!\n\x14author_career_job1_3\x12\t\n\x07\n\x0511075\n\x10\n\x07max_len\x12\x05\x1a\x03\n\x01\x14\n\x14\n\x0blike_number\x12\x05\x1a\x03\n\x01\n\n\x12\n\tcvr_label\x12\x05\x1a\x03\n\x01\x00\n\x0e\n\x05label\x12\x05\x1a\x03\n\x01\x01\n \n\x13manual_career_job_2\x12\t\n\x07\n\x0511225\n\x0e\n\x05ctr_3\x12\x05\x1a\x03\n\x01\x07\n\x15\n\x0ccontent_mode\x12\x05\x1a\x03\n\x01\x02\n \n\x17company_keyword_max_len\x12\x05\x1a\x03\n\x01\x01\n\x1a\n\rcareer_job1_1\x12\t\n\x07\n\x0511227\n\x14\n\x0bpost_module\x12\x05\x1a\x03\n\x01\x00\n\r\n\x04form\x12\x05\x1a\x03\n\x01\x00\n.\n\rskill_keyword\x12\x1d\n\x1b\n\x0b10000000000\n\x05<nan>\n\x05<nan>\n \n\x13manual_career_job_1\x12\t\n\x07\n\x0511231\n\x1a\n\rcareer_job1_3\x12\t\n\x07\n\x0511043\n\x1b\n\x12work_status_detail\x12\x05\x1a\x03\n\x01\x01\n3\n\x13author_school_major\x12\x1c\n\x1a\n\x18\xe8\x88\xb9\xe8\x88\xb6\xe7\x94\xb5\xe5\xad\x90\xe7\x94\xb5\xe6\xb0\

2024-06-26 17:35:44.248206: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## dataset.take
首先 dataset.take后返回的也是一个可遍历的 dataset，所以我们用[] = 来接，这样 feature 取的就是真实的 dataset 的第一个数据
feature是一个 dict，key 是特征的名称，value 是特征值，类别类型(1,)，可以理解成一个样本一个 string 之类的

In [56]:
dataset = dataset.map(_parse_example, num_parallel_calls=8)
[feature] = dataset.take(1)
print(feature)
print("----------------------------")
print(feature['school_type'])

{'CES': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, 'author_career_job1_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'11229'], dtype=object)>, 'author_career_job1_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'11210'], dtype=object)>, 'author_career_job1_3': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'11075'], dtype=object)>, 'author_edu_level': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'\xe5\x85\xb6\xe4\xbb\x96'], dtype=object)>, 'author_edu_work_status': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>, 'author_gender': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'\xe5\x85\xb6\xe4\xbb\x96'], dtype=object)>, 'author_school': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'\xe5\xb1\xb1\xe4\xb8\x9c\xe4\xba\xa4\xe9\x80\x9a\xe5\xad\xa6\xe9\x99\xa2'],
      dtype=object)>, 'author_school_major': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'\xe8\x88\xb9\xe8\x88\xb6\xe7\x94\xb5\xe5\xad\x90\xe7\

2024-06-26 17:35:44.318112: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## batch(3)
看起来是，原本(1,)的数据变成了(3,1)，这是将原来3个样本数据 [1],[2],[3]合并成一个 tensor，[[1],[2],[3]]
，至于为什么呢？看 gpt 的解读
<img src="../../data/img/dataset_take.png"/>


In [57]:
dataset = dataset.batch(3)
[feature] = dataset.take(1)
print(feature)
print("----------------------------")
print(feature['school_type'])

{'CES': <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>, 'author_career_job1_1': <tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b'11229'],
       [b'11226'],
       [b'11226']], dtype=object)>, 'author_career_job1_2': <tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b'11210'],
       [b'11200'],
       [b'11200']], dtype=object)>, 'author_career_job1_3': <tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b'11075'],
       [b'11002'],
       [b'11003']], dtype=object)>, 'author_edu_level': <tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b'\xe5\x85\xb6\xe4\xbb\x96'],
       [b'\xe5\xad\xa6\xe5\xa3\xab'],
       [b'\xe5\xad\xa6\xe5\xa3\xab']], dtype=object)>, 'author_edu_work_status': <tf.Tensor: shape=(3, 1), dtype=int64, numpy=
array([[1],
       [1],
       [1]])>, 'author_gender': <tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b'\xe5\x85\xb6\xe4\xbb\x96'],
       [b'\xe5\x85\xb6\xe4\xbb\x96'],
   

2024-06-26 17:35:44.352916: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [58]:
# 这里就很怪诞了，batch 后又 batch
dataset = dataset.batch(1)
[feature] = dataset.take(1)
print(feature)
print("----------------------------")
print(feature['school_type'])

{'CES': <tf.Tensor: shape=(1, 3, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [1.]]], dtype=float32)>, 'author_career_job1_1': <tf.Tensor: shape=(1, 3, 1), dtype=string, numpy=
array([[[b'11229'],
        [b'11226'],
        [b'11226']]], dtype=object)>, 'author_career_job1_2': <tf.Tensor: shape=(1, 3, 1), dtype=string, numpy=
array([[[b'11210'],
        [b'11200'],
        [b'11200']]], dtype=object)>, 'author_career_job1_3': <tf.Tensor: shape=(1, 3, 1), dtype=string, numpy=
array([[[b'11075'],
        [b'11002'],
        [b'11003']]], dtype=object)>, 'author_edu_level': <tf.Tensor: shape=(1, 3, 1), dtype=string, numpy=
array([[[b'\xe5\x85\xb6\xe4\xbb\x96'],
        [b'\xe5\xad\xa6\xe5\xa3\xab'],
        [b'\xe5\xad\xa6\xe5\xa3\xab']]], dtype=object)>, 'author_edu_work_status': <tf.Tensor: shape=(1, 3, 1), dtype=int64, numpy=
array([[[1],
        [1],
        [1]]])>, 'author_gender': <tf.Tensor: shape=(1, 3, 1), dtype=string, numpy=
array([[[b'\xe5\x85\xb6\xe4\xbb\x9

2024-06-26 17:35:44.387051: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
